In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TabNet 模型的 Vertex AI 解释

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/ai-explanations-tabnet-algorithm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/ai-explanations-tabnet-algorithm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabnet/ai-explanations-tabnet-algorithm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

Vertex AI 提供了一种名为[TabNet](https://arxiv.org/abs/1908.07442)的算法。TabNet是一种用于表格（结构化）数据的可解释深度学习架构，是企业中最常见的数据类型。TabNet结合了两个世界的优势：它具有可解释性，就像简单的基于树的模型，同时又可以实现复杂黑盒模型和集成模型的高准确性，意味着它既精确又不会模糊模型的工作原理。这使得TabNet非常适合一系列表格数据任务，其中模型的可解释性和准确性同样重要。

本教程的目标是提供一个示例绘图工具，用于可视化TabNet的输出，从而有助于解释该算法。

了解更多关于[Tabular Workflow for TabNet](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/tabnet)。

### 目标

在本教程中，您将学习如何提供一个示例绘图工具来可视化TabNet的输出，这有助于解释该算法。

本教程使用以下Google Cloud ML服务和资源：

- Vertex Explainable AI
- TabNet内置算法

所执行的步骤包括：
* 设置项目。
* 下载在Syn2数据上预训练模型的预测数据。
* 根据掩码输出可视化并理解特征重要性。
* 清理本教程创建的资源。

### 数据集

本教程使用合成数据集Synthetic_2 (Syn2)，详细描述在 [学习解释](https://arxiv.org/pdf/1802.07814.pdf) 论文的第4.1节中。输入特征 X 是从一个10维标准高斯分布生成的。响应变量 Y 仅从特征 X[3:6] 中生成。数据已分为训练集和预测集，并已上传至Google Cloud Storage:
* 训练数据: gs://cloud-samples-data/ai-platform-unified/datasets/tabnet/tab_net_input/syn2_train.csv。
* 预测输出数据: gs://cloud-samples-data/ai-platform-unified/datasets/tabnet/tab_net_output/syn2

目前，TabNet预训练模型文件尚未公开提供。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您预计的使用量生成成本估算。

安装以下必要的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform tensorflow

### 只有在 Colab 上：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置你的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个帐户时，您会获得 $300 的免费信用用于支付您的计算/存储费用。

2. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下 API：Vertex AI API，Cloud 资源管理器 API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK（云 SDK）](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`。
- 运行 `gcloud projects list`。
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享测试帐户或项目。为了避免在创建的资源之间发生名称冲突，您为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照下面的相关说明进行操作。

**1. Vertex AI Workbench**
* 不用做任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
* 请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上向您的服务帐户授予云存储权限。

创建一个云存储桶 

创建一个存储桶来存储诸如数据集之类的中间产物。

当您使用云 SDK 提交训练作业时，您会将包含您的训练代码的 Python 软件包上传到云存储桶中。Vertex AI 会运行来自这个软件包的代码。在此教程中，Vertex AI 也会将您的作业产生的训练模型保存在同一个存储桶中。利用这个模型产物，您可以创建 Vertex AI 模型资源并用于预测。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才会执行：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库并定义常量

In [ ]:
import json

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import storage

%matplotlib inline

## 在syn2数据上阅读一个样本TabNet预测

在训练并提供您的模型之后，您将上传输出到Google Cloud Storage。

样本预测数据存储在Google Cloud上的gs://cloud-samples-data/ai-platform-unified/datasets/tabnet/tab_net_output/syn2。您可以使用自己的预测数据集，但必须确保预测数据的格式与训练数据的格式相同。

TabNet中的每个预测都包含一个用于解释预测的掩码。该掩码存储在**aggregated_mask_values**字段中。

有关训练集和模型的信息最适合在笔记本的上一部分的数据集部分中查看。

In [ ]:
!gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabnet/tab_net_output/syn2 $BUCKET_URI

# Replace your the BUCKET_URI and PREDICTION_FILE
# BUCKET_NAME = "[<your-bucket-name>]"
# PREDICTION_FILE = "[<your-prediction-file>]"

BUCKET_NAME = BUCKET_URI[5:]
PREDICTION_FILE = "syn2"

MASK_KEY = "aggregated_mask_values"

HEADER = [("feat_" + str(i)) for i in range(1, 12)]
HEADER

### 下载并预处理预测数据。

In [ ]:
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)
blob = bucket.blob(PREDICTION_FILE)
f = blob.download_as_string(client=None).decode("utf-8").strip()
predictions = f.split("\n")
predictions[:1]

## 解析预测中的掩码值。然后，连接掩码值。输出是一个具有Nxk（N为输出数量，k为每个掩码的大小）的矩阵。连接掩码值用于可视化特征重要性。

In [ ]:
masks = []
for prediction in predictions:
    prediction = json.loads(prediction)
    masks.append(prediction[MASK_KEY])
masks = np.matrix(masks)
masks.shape

将蒙版数值矩阵可视化。
浅色表示更重要的特征。例如，在Syn2数据中，只有特征3-6在预测输出中是有意义的。在图中，列3-6呈现浅色。

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(121)
ax.imshow(masks[:50, :], interpolation="bilinear", cmap=cm.Greys_r)
ax.set_xlabel("Features")
ax.set_ylabel("Sample index")
ax.xaxis.set_ticks(np.arange(len(HEADER)))
ax.set_xticklabels(HEADER, rotation="vertical")
plt.show()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

# Delete Cloud Storage that were created
if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

接下来是什么？

要了解更多关于TabNet的信息，请查看这里的资源。

* [TabNet：注重可解释的表格学习](https://arxiv.org/abs/1908.07442)